### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2023-11-13 07:54:52--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2023-11-13 07:54:53--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8805d9f17c744ab9c8a6b18a81.dl.dropboxusercontent.com/cd/0/get/CHeSW_sWE6PwxALEXXs-tQVbBs9EVbAVvQFv1dHovkpY3_QxVtZ2ftp4qP7Kvj2Jv-asMuK5KJjrUsSChnFWuV-6VpU8s8jwnnlksR7qeF1h-Fz_PE74s4UbsW087wI-MxU/file?dl=1# [following]
--2023-11-13 07:54:53--  https://uc8805d9f17c744ab9c8a6b18a81.dl.dropboxusercontent.com/cd/0/get/CHeSW_sWE6PwxALEXXs-tQVbBs9EVbAVvQFv1dHovkpY3_QxVtZ2ftp4qP7Kvj2Jv-asMuK5KJjrUsSChnFWuV-6VpU

,author,day,id,link,month,summary,tag,title,year
23213,"[{'name': 'Yuya Inagaki'}, {'name': 'Jun-ichi ...",21,1004.3629v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We numerically investigate a mean-field Bayesi...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Simultaneous Bayesian inference of motion velo...,2010
30469,"[{'name': 'Zhe Hu'}, {'name': 'Yinglan Ma'}, {...",7,1712.02874v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Traditional approaches to interpolate/extrapol...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Multi-Scale Video Frame-Synthesis Network with...,2017
13298,"[{'name': 'Fabrice Rossi'}, {'name': 'Brieuc C...",2,0802.0251v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,"In some real world situations, linear models a...","[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Multi-Layer Perceptrons and Symbolic Data,2008
39310,"[{'name': 'Giulia Pasquale'}, {'name': 'Tanis ...",23,1509.06939v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,The importance of depth perception in the inte...,"[{'term': 'cs.RO', 'scheme': 'http://arxiv.org...",Enabling Depth-driven Visual Attention on the ...,2015
38612,[{'name': 'Jason L. Williams'}],14,1203.2992v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,The probability hypothesis density (PHD) and m...,"[{'term': 'cs.SY', 'scheme': 'http://arxiv.org...",Hybrid Poisson and multi-Bernoulli filters,2012


In [ ]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [ ]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

for i in range(len(lines)):
    lines[i] = ' '.join(tokenizer.tokenize(lines[i].lower()))

In [ ]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [ ]:
from prompt_toolkit.shortcuts.progress_bar.base import E
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for text in lines:
        text += ' ' + EOS
        for i in range(len(text.split())):

            #calculate prefixes

            if (i-n) < 0:
                prefix = tuple(UNK for _ in range(n-i-1)) + tuple(x for x in text.split()[:i]) #prefixes filling with UKN values (if tokens index < our n-gramm)
            else:
                prefix = tuple(x for x in text.split()[i-n+1:i])

            #filling our conts dict

            if prefix not in counts:
                counts[prefix] = {text.split()[i]:1}

            else:
                if text.split()[i] not in counts[prefix]:
                    counts[prefix][text.split()[i]] = 1
                else:
                    counts[prefix][text.split()[i]] +=1


    return counts


In [ ]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [ ]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix in counts:
            full_sum = sum([counts[prefix][word] for word in counts[prefix]])
            for word in counts[prefix]:
                self.probs[prefix][word] = counts[prefix][word] / full_sum

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [ ]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [ ]:
lm = NGramLanguageModel(lines, n=3)

In [ ]:
prefix = "to improve the perplexity of a language model , one should"

print(end=prefix + ' ', flush=True)
for i in range(100):
    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())
    next_token = np.random.choice(tokens, p=probs)
    prefix = prefix + ' ' + next_token
    print(end=next_token + ' ', flush=True)

to improve the perplexity of a language model , one should be evaluated fast using only small convolutions and to recommend points to two or more languages and the corresponding sparse codes are then converted into the recently released dataset ( 245 gb ) has seen huge adoption in computer vision . however , pixel intensities , the spatial path signature ( lnps ), and outperforms purely model - based sparse pursuit problem and obtain better `` high - dimensional structure of polynomial size certificates for it . in this paper proposes a model , and energy consumption behavior at the same ansig . we show that training the selected sentences 

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    delta = 0.01

    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())

    total_probs = sum([prob**(1/(temperature+delta)) for prob in probs])

    final_probs = [(prob)**(1/(temperature+delta))/total_probs for prob in probs]

    #adding small delta to bypass division by zero

    return np.random.choice(tokens, p=final_probs)

In [ ]:
lm.get_possible_next_tokens('there have')

Counter({'been': 0.9083969465648855,
         'not': 0.03816793893129771,
         'only': 0.015267175572519083,
         'also': 0.015267175572519083,
         'lately': 0.007633587786259542,
         'very': 0.007633587786259542,
         'occurred': 0.007633587786259542})

In [ ]:
get_next_token(lm, 'deep', temperature=0.0)

'learning'

In [ ]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [ ]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial brain signals . this paper , we present a brief literature review : this paper , we show that the proposed network is point - level tasks . however , has achieved impressive improvements achieved by allowing for a longer sequence of words which are hierarchical in nature to generative modeling ; randomized search heuristics and compare the player ' s commonalities for the reward distributions . to address the problem to the spectral kernel distance results from our extensive experiments demonstrate that the herding algorithm tends to attribute correctly . next , we present translations from mt engines for


In [ ]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - dimensional , non - smooth functions using machine learning . we demonstrate that our method is an important tool in large - scale problems . however , the performance of our method can be used to compute the facts they explicitly know . we investigate the performance of the most important and challenging research topic . we show how this approach is based on the evaluation of the proposed method is found to be used to develop a decision - making problem . we evaluate our approach is the first and second - order methods


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

$$
    {\ln{\mathbb{P}}}(w_1 \dots w_N) = \ln \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N} = - \frac{1}{N} \sum_t \ln{P(w_t \mid w_{t - n}, \dots, w_{t - 1}})
$$

In [ ]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -30.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    perplex = 0
    counter = 0
    for line in lines:
        prefix =  line.split()
        for i in range(len(prefix)):
            if i > lm.n:
                prob = np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[i-lm.n:i]), prefix[i])), min_logprob)
            else:
                if i != 0:
                    prob = np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[:i]), prefix[i])), min_logprob)
                else:
                    prob = np.maximum(np.log(lm.get_next_token_prob('', prefix[0])), min_logprob) #empty prefix = zero token
            perplex += prob
            counter += 1
        perplex += np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[-lm.n:]), EOS)), min_logprob) #last token - __EOS__
        counter += 1

    return np.exp( (-1 / counter) * perplex)

In [ ]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


<ipython-input-321-3603ca54a155>:20: RuntimeWarning: divide by zero encountered in log
  prob = np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[:i]), prefix[i])), min_logprob)
<ipython-input-321-3603ca54a155>:17: RuntimeWarning: divide by zero encountered in log
  prob = np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[i-lm.n:i]), prefix[i])), min_logprob)
<ipython-input-321-3603ca54a155>:25: RuntimeWarning: divide by zero encountered in log
  perplex += np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[-lm.n:]), EOS)), min_logprob) #last token - __EOS__


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


<ipython-input-321-3603ca54a155>:17: RuntimeWarning: divide by zero encountered in log
  prob = np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[i-lm.n:i]), prefix[i])), min_logprob)
<ipython-input-321-3603ca54a155>:22: RuntimeWarning: divide by zero encountered in log
  prob = np.maximum(np.log(lm.get_next_token_prob('', prefix[0])), min_logprob) #empty prefix = zero token
<ipython-input-321-3603ca54a155>:20: RuntimeWarning: divide by zero encountered in log
  prob = np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[:i]), prefix[i])), min_logprob)


N = 1, Perplexity = 1314.48051


<ipython-input-321-3603ca54a155>:25: RuntimeWarning: divide by zero encountered in log
  perplex += np.maximum(np.log(lm.get_next_token_prob(' '.join(prefix[-lm.n:]), EOS)), min_logprob) #last token - __EOS__


N = 2, Perplexity = 269663.32619
N = 3, Perplexity = 88071875073213.23438


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765
